In [ ]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import os
import cv2 as cv
import random 
import argparse

Definition der Datensatz Klasse, diese stellt Bilder + Annotationen zum training bereit. Die Bilder werden dabei zur trainingszeit zusammen gesetzt sodass eine Explizite größen angabe erfolgen muss.

In [ ]:
class fake_dataset(Dataset):
    def __init__(self, sign_path: str, background_path: str, length, scr):
        self.sign_path = sign_path
        self.background_path = background_path
        self.signs = os.listdir(sign_path)
        self.backgrounds = os.listdir(background_path)
        self.length = length
        self.scr = scr
    def get_rand_img(root, paths):
        path = paths[random.randrange(0, len(paths))]
        return cv.imread(os.path.join(root, path))
    def __len__(self):
        return self.length
    def __getitem__(self, idx):
        sign_count = random.randrange(self.scr[0], self.scr[1])
        

In [ ]:
ds = fake_dataset
ds = fake_dataset
train_loader = DataLoader(ds, 4)
eval_loader = DataLoader(ds, 4)

model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn()
#@todo Fix Hardcoded params
parameters = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
            parameters,
            lr=0.01,
            momentum=1,
            weight_decay=1,
        )

for i in range(100):
    for images, targets in train_loader:
        result = model(images, targets)
    #with torch.cuda.amp.autocast(enabled=scaler is not None):
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()